In [1]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct

%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

from collections import defaultdict
from tqdm import *

data_dir = '/mnt/data/cdiscount/'

from subprocess import check_output
print(check_output(["ls", data_dir]).decode("utf8"))

num_blocks = 25
# Any results you write to the current directory are saved as output.

category_names.csv
sample_submission.csv
test.bson
train_block_0.bson
train_block_10.bson
train_block_11.bson
train_block_12.bson
train_block_13.bson
train_block_14.bson
train_block_15.bson
train_block_16.bson
train_block_17.bson
train_block_18.bson
train_block_19.bson
train_block_1.bson
train_block_20.bson
train_block_21.bson
train_block_22.bson
train_block_23.bson
train_block_24.bson
train_block_2.bson
train_block_3.bson
train_block_4.bson
train_block_5.bson
train_block_6.bson
train_block_7.bson
train_block_8.bson
train_block_9.bson
train.bson
train_example.bson



In [2]:
train_block_bson_path_list = []
for i in range(num_blocks):
    train_block_bson_filename = 'train_block_' + str(i) + '.bson'
    train_block_bson_path_list.append(os.path.join(data_dir, train_block_bson_filename))

num_train_block_products = 1413980

In [3]:
categories_df = pd.read_csv("categories.csv", index_col=0)
def make_category_tables():
    cat2idx = {}
    idx2cat = {}
    for ir in categories_df.itertuples():
        category_id = ir[0]
        category_idx = ir[4]
        cat2idx[category_id] = category_idx
        idx2cat[category_idx] = category_id
    return cat2idx, idx2cat

cat2idx, idx2cat = make_category_tables()
cat2idx[1000012755], idx2cat[4]

(4, 1000012755)

In [4]:
def read_bson(bson_path, num_records, with_categories):
    rows = {}
    with open(bson_path, "rb") as f, tqdm(total=num_records) as pbar:
        offset = 0
        while True:
            item_length_bytes = f.read(4)
            if len(item_length_bytes) == 0:
                break

            length = struct.unpack("<i", item_length_bytes)[0]

            f.seek(offset)
            item_data = f.read(length)
            assert len(item_data) == length

            item = bson.BSON.decode(item_data)
            product_id = item["_id"]
            num_imgs = len(item["imgs"])

            row = [num_imgs, offset, length]
            if with_categories:
                row += [item["category_id"]]
            rows[product_id] = row

            offset += length
            f.seek(offset)
            pbar.update()

    columns = ["num_imgs", "offset", "length"]
    if with_categories:
        columns += ["category_id"]

    df = pd.DataFrame.from_dict(rows, orient="index")
    df.index.name = "product_id"
    df.columns = columns
    df.sort_index(inplace=True)
    return df

train_block_offsets_df_list = []
for i in range(num_blocks):
    train_block_bson_filename = 'train_block_' + str(i) + '.bson'
    train_block_bson_path = os.path.join(data_dir, train_block_bson_filename)
    train_block_offsets_df_list.append(read_bson(train_block_bson_path, num_records=num_train_block_products, with_categories=True))

 20%|█▉        | 282794/1413980 [00:08<00:32, 34279.57it/s]


In [5]:
for i, df in enumerate(train_block_offsets_df_list):
    offset_csv_filename = 'train_block_' + str(i) + '_offsets.csv'
    df.to_csv(offset_csv_filename)

In [6]:
for i, df in enumerate(train_block_offsets_df_list):
    print('block %d' % i)
    print('number of products = %d' % len(df))
    print('number of catagories = %d' % len(df["category_id"].unique()))
    print('number of images = %d' % df["num_imgs"].sum())

block 0
number of products = 282796
number of catagories = 4882
number of images = 495389
block 1
number of products = 282796
number of catagories = 4901
number of images = 494121
block 2
number of products = 282796
number of catagories = 4880
number of images = 494809
block 3
number of products = 282796
number of catagories = 4872
number of images = 494692
block 4
number of products = 282796
number of catagories = 4915
number of images = 494378
block 5
number of products = 282796
number of catagories = 4877
number of images = 495252
block 6
number of products = 282796
number of catagories = 4900
number of images = 494569
block 7
number of products = 282796
number of catagories = 4879
number of images = 494409
block 8
number of products = 282796
number of catagories = 4873
number of images = 494617
block 9
number of products = 282796
number of catagories = 4907
number of images = 495297
block 10
number of products = 282796
number of catagories = 4886
number of images = 495392
block 11


In [7]:
def make_val_set(df, split_percentage=0.2, drop_percentage=0.):
    # Find the product_ids for each category.
    category_dict = defaultdict(list)
    for ir in tqdm(df.itertuples()):
        category_dict[ir[4]].append(ir[0])

    train_list = []
    val_list = []
    with tqdm(total=len(df)) as pbar:
        for category_id, product_ids in category_dict.items():
            category_idx = cat2idx[category_id]

            # Randomly remove products to make the dataset smaller.
            keep_size = int(len(product_ids) * (1. - drop_percentage))
            if keep_size < len(product_ids):
                product_ids = np.random.choice(product_ids, keep_size, replace=False)

            # Randomly choose the products that become part of the validation set.
            val_size = int(len(product_ids) * split_percentage)
            if val_size > 0:
                val_ids = np.random.choice(product_ids, val_size, replace=False)
            else:
                val_ids = []

            # Create a new row for each image.
            for product_id in product_ids:
                row = [product_id, category_idx]
                for img_idx in range(df.loc[product_id, "num_imgs"]):
                    if product_id in val_ids:
                        val_list.append(row + [img_idx])
                    else:
                        train_list.append(row + [img_idx])
                pbar.update()
                
    columns = ["product_id", "category_idx", "img_idx"]
    train_df = pd.DataFrame(train_list, columns=columns)
    val_df = pd.DataFrame(val_list, columns=columns)   
    return train_df, val_df

train_block_images_df_list = []
val_block_images_df_list = []

for offsets_df in train_block_offsets_df_list:
    train_block_images_df, val_block_images_df = make_val_set(offsets_df, split_percentage=0.2, drop_percentage=0.)
    train_block_images_df_list.append(train_block_images_df)
    val_block_images_df_list.append(val_block_images_df)

282796it [00:00, 667440.79it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55831.64it/s]
282796it [00:00, 776933.29it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55463.11it/s]
282796it [00:00, 773074.21it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55144.97it/s]
282796it [00:00, 768736.27it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55267.37it/s]
282796it [00:00, 777024.39it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55815.43it/s]
282796it [00:00, 770582.62it/s]
100%|██████████| 282796/282796 [00:05<00:00, 54878.67it/s]
282796it [00:00, 774627.70it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55975.11it/s]
282796it [00:00, 774993.12it/s]
100%|██████████| 282796/282796 [00:05<00:00, 56435.18it/s]
282796it [00:00, 774627.19it/s]
100%|██████████| 282796/282796 [00:04<00:00, 56842.06it/s]
282796it [00:00, 773907.48it/s]
100%|██████████| 282796/282796 [00:05<00:00, 56022.03it/s]
282796it [00:00, 769783.96it/s]
100%|██████████| 282796/282796 [00:05<00:00, 55471.96it/s]

In [9]:
i = 0
for train_block_images_df, val_block_images_df in zip(train_block_images_df_list, val_block_images_df_list):
    train_block_image_csv_filename = 'train_block_' + str(i) + '_images.csv'
    val_block_image_csv_filename = 'val_block_' + str(i) + '_images.csv'
    train_block_images_df.to_csv(train_block_image_csv_filename)
    val_block_images_df.to_csv(val_block_image_csv_filename)
    i = i + 1